In [2]:
!nvidia-smi

Sun Mar 19 21:22:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    28W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!pip install datasets transformers rouge-score nltk py7zr
!pip install wandb -qq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 84.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 KB 374.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 88.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 K

In [5]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
%env WANDB_PROJECT=bart_large_xsum_samsum

env: WANDB_PROJECT=bart_large_xsum_samsum


In [7]:
model_checkpoint = "facebook/bart-large-xsum"

## Loading the dataset

In [8]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("samsum")
metric = load_metric("rouge")

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Dataset samsum downloaded and prepared to /root/.cache/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

<ipython-input-8-a2ef1525156b>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


## Preprocessing the data

In [9]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [10]:
max_input_length = 1024
max_target_length = 512

def preprocess_function(examples):
    inputs = [doc for doc in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [11]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3586: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

## Fine-tuning the model

In [12]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [13]:
batch_size = 4
args = Seq2SeqTrainingArguments(
    "summarization",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    report_to="wandb",
    run_name="bart-large-xsum-samsum"
)

In [14]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [15]:
import nltk
import numpy as np
nltk.download('punkt')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [16]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [18]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


wandb: Currently logged in as: atish-adhikari. Use `wandb login --relogin` to force relogin


You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,1.384100,1.483374,53.184400,28.720700,44.037800,48.888500,26.469400
2,1.077000,1.441497,54.735200,29.380100,44.645900,50.224200,31.524400
2,0.856000,1.480368,54.480200,29.614300,44.911800,50.120900,30.501200


TrainOutput(global_step=5523, training_loss=1.1183089716115464, metrics={'train_runtime': 4417.0162, 'train_samples_per_second': 10.006, 'train_steps_per_second': 1.25, 'total_flos': 2.609313301001011e+16, 'train_loss': 1.1183089716115464, 'epoch': 3.0})

In [19]:
trainer.evaluate()

{'eval_loss': 1.4803681373596191,
 'eval_rouge1': 54.4802,
 'eval_rouge2': 29.6143,
 'eval_rougeL': 44.9118,
 'eval_rougeLsum': 50.1209,
 'eval_gen_len': 30.5012,
 'eval_runtime': 334.0814,
 'eval_samples_per_second': 2.449,
 'eval_steps_per_second': 0.614,
 'epoch': 3.0}

In [20]:
trainer.save_model('saved_modal')

In [21]:
wandb.finish()

eval/gen_len,▁█▇▇
eval/loss,█▁▇▇
eval/rouge1,▁█▇▇
eval/rouge2,▁▆██
eval/rougeL,▁▆██
eval/rougeLsum,▁█▇▇
eval/runtime,▁▃▃█
eval/samples_per_second,█▅▆▁
eval/steps_per_second,█▅▆▁
train/epoch,▁▂▂▃▃▄▄▅▅▆▇▇████
train/global_step,▁▂▂▃▃▄▄▅▅▆▇▇████


In [30]:
from transformers import pipeline

summarizer = pipeline("summarization", model="saved_modal")
conversation = '''Question: What is your initials?
Answer: AK
Question: What is your gender?
Answer: Woman
Question: How old are you?
Answer: 36 - 50
Question: Please select your current country, State (Province) of residence.
Answer: Canada, Quebec
Question: What is your occupation?
Answer: Work
Question: What is your annual income?
Answer: > $60000
Question: Have you ever been diagnosed with a mental health condition?
(DSM 5 defines a mental health condition as a clinical manifestation of an individual's behavioral or emotional patterns)
Answer: No
Question: Have you ever noticed any behavioral or emotional patterns?
Answer: Anxiety
Question: When was the last time you have experienced symptoms of Anxiety?
Answer: < 1 year
Question: Are you currently seeking any treatment for your Anxiety?
Answer: No
Question: How did the therapist approach support you in your therapeutic process?
Answer: Moderate
Question: How did it affect your day to day life?
Answer: Prone to more isolation;
Question: What specific goals you would like to achieve through treatment for your Anxiety?
Answer: To feel better at work or school;To be happy;To have better relationships;
Question: What are your expectation from that treatment?
Answer: To feel relax;To learn coping strategies;To have better relationships;To feel better at work or school;To be happy;
Question: In general, have you noticed any changes in your relationships with family, friends, workplace or other?
Answer: Mostly at the workplace;
Question: How does your sleep quality and quantity impact your mood and daily functioning?
Answer: Moderately
Question: Has it affected any change in your sleep patterns?
Answer: Moderately
Question: Has there been any sudden changes in your overall mood? 
Answer: Moderately
Question:  What kind of changes have you experienced?
Answer: Overwhelmed/Stimulated;
Question: In general what else was affected by your Anxiety symptoms?
Answer: Long focus and attention to some tasks
Question: Have you experienced any financial or employment difficulties as a result of your Anxiety symptoms?
Answer: A little bit
Question: If you have a chance to use self-guided real-life scenario to overcome your anxiety would you use it?
Answer: Yes, I would use it monthly
Question: How do you cope with triggers or symptoms of Anxiety on a daily basis?
Answer: mediation                                 
'''
summarizer(conversation)

[{'summary_text': 'A woman from Canada, aged 36-50, lives in Quebec and earns over $60000 per year. She suffers from anxiety. She is seeing a therapist for her anxiety.'}]